In [1]:
# -*- coding: utf-8 -*-
import glob, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
"""
Taken from https://www.kaggle.com/code/hashimo227/car-ecu-datalogs-scatter-plot?scriptVersionId=45999601&cellId=5
"""

input_files = glob.glob("../input/car-ecu-datalogs/2018*.csv", recursive=True)
all_data = None
for i in range(2):
    csv_columns = []
    csv_columns.append('time')
    header_number = 0
    channel = ""
    with open(input_files[i]) as f:
        lines = f.readlines()
        for line in lines:
            header_number += 1
            if line.startswith('Channel : '):
                channel = line.replace('Channel : ','').replace('\n','')
            if line.startswith('Type : '):
                csv_columns.append(channel + "[" + line.replace('Type : ','').replace('\n','') + "]")
            if line.startswith('Log : '):
                break
#     print("{} header={}  file={}".format(i+1, header_number, input_files[i]))
    df = pd.read_csv(input_files[i], index_col=False, skiprows=header_number, names=csv_columns)
    
    basename = os.path.basename(input_files[i])
    df['date'] = basename.replace('.csv','').split('-')[0]
    route = basename.replace('.csv','').split('-')[1]
    df['route'] = route
    df['time'] = df['date'] + " " + df['time']
    df['time'] = pd.to_datetime( df['time'], format="%Y%m%d %H:%M:%S.%f")
    df['#time_diff'] = df['time'].diff(1).dt.total_seconds()
    df['#time_seq'] = df['#time_diff'].cumsum()
    df['#road_seq'] = df['#time_seq']
    if route == "mimos2home":
        df['#road_seq'] = df['#road_seq'].max() - df['#road_seq']

    if all_data is None:
        all_data = df
    else:
        all_data = pd.concat([all_data, df])

In [3]:
for i in all_data.columns:
    print(i)

time
AFRDifference[AFR]
BatteryVoltage[BatteryVoltage]
EngineRunningTime[Time_s]
IgnitionTiming[Angle]
CurrentDutyCycle[Percentage]
Load[Pressure]
IgnitionLoad[Pressure]
ShiftLight1[Raw]
AirTemp[Temperature]
CoolantTemp[Temperature]
MAPSource[Pressure]
LambdaSensor1[AFR]
ThrottlePosition[Percentage]
BaseFuel[Percentage]
BaseIgnition[AngleIgnSprt2K]
TargetAFR[AFR]
FuelAirTempCorrection[Percentage]
FuelCoolantTempCorrection[Percentage1For1]
IgnitionCoolantTempCorrection[AngleOffset10deg]
IgnitionAirTempCorrection[Angle]
RPM[EngineSpeed]
TransientThrottleAccelCoolantCorr[Percentage]
TransientThrottleEnrichSensitivity[Time_us]
TransientThrottleEnrichDecayRate[msPerEngCyl]
TransientThrottlePercentageAsync[Percentage]
TransientThrottlePercentageEnrich[Percentage]
Pwm_State_2[Raw]
Pwm_State_1[Raw]
date
route
#time_diff
#time_seq
#road_seq


In [4]:
slimmed_data = all_data[[    "time","AFRDifference[AFR]","IgnitionTiming[Angle]","CurrentDutyCycle[Percentage]",
                             "Load[Pressure]", "AirTemp[Temperature]","CoolantTemp[Temperature]","LambdaSensor1[AFR]", "ThrottlePosition[Percentage]",
                             "TargetAFR[AFR]", "RPM[EngineSpeed]"
                        ]]

In [5]:
print(slimmed_data.head())

                     time  AFRDifference[AFR]  IgnitionTiming[Angle]  \
0 2018-08-20 06:17:34.000                28.0                  160.0   
1 2018-08-20 06:17:34.047                 NaN                    NaN   
2 2018-08-20 06:17:34.078                 NaN                    NaN   
3 2018-08-20 06:17:34.094                 NaN                    NaN   
4 2018-08-20 06:17:34.203                 NaN                    NaN   

   CurrentDutyCycle[Percentage]  Load[Pressure]  AirTemp[Temperature]  \
0                          23.0           397.0                3132.0   
1                           NaN           396.0                   NaN   
2                           NaN           397.0                   NaN   
3                          23.0             NaN                   NaN   
4                           NaN             NaN                   NaN   

   CoolantTemp[Temperature]  LambdaSensor1[AFR]  ThrottlePosition[Percentage]  \
0                    3257.0                 0.0